# pandas time series analysis

In [5]:
import pandas as pd
df = pd.read_csv('aapl_no_dates_15.csv')
df.head()

,Open,High,Low,Close,Volume
0,144.88,145.30,143.10,143.50,14277848
1,143.69,144.79,142.72,144.09,21569557
2,143.02,143.50,142.41,142.73,24128782
3,142.90,144.75,142.90,144.18,19201712
4,144.11,145.95,143.37,145.06,21090636


In [7]:
pd.date_range(start='7-1-2017',end='7-21-2017',freq='B')

DatetimeIndex(['2017-07-03', '2017-07-04', '2017-07-05', '2017-07-06',
               '2017-07-07', '2017-07-10', '2017-07-11', '2017-07-12',
               '2017-07-13', '2017-07-14', '2017-07-17', '2017-07-18',
               '2017-07-19', '2017-07-20', '2017-07-21'],
              dtype='datetime64[ns]', freq='B')

In [9]:
# now 4 july is holiday in US and stocks do not spray that day
# in our csv file(with date) there is not stock for 4th july

Using 'B' frequency is not going to help because 4th July was holiday and 'B' is not taking that into account. It only accounts for weekends

# Using CustomBusinessDay to generate US holidays calendar frequency

In [11]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

usb = CustomBusinessDay(calendar=USFederalHolidayCalendar())
usb

<CustomBusinessDay>

In [12]:
pd.date_range(start='7-1-2017',end='7-21-2017',freq=usb) # exclude weekwnd and holiday

DatetimeIndex(['2017-07-03', '2017-07-05', '2017-07-06', '2017-07-07',
               '2017-07-10', '2017-07-11', '2017-07-12', '2017-07-13',
               '2017-07-14', '2017-07-17', '2017-07-18', '2017-07-19',
               '2017-07-20', '2017-07-21'],
              dtype='datetime64[ns]', freq='C')

In [13]:
rng = pd.date_range(start='7-1-2017',end='7-21-2017',freq=usb)


In [15]:
df.set_index(rng,inplace = True)
df

,Open,High,Low,Close,Volume
2017-07-03,144.88,145.30,143.10,143.50,14277848
2017-07-05,143.69,144.79,142.72,144.09,21569557
2017-07-06,143.02,143.50,142.41,142.73,24128782
2017-07-07,142.90,144.75,142.90,144.18,19201712
2017-07-10,144.11,145.95,143.37,145.06,21090636
2017-07-11,144.73,145.85,144.38,145.53,19781836
2017-07-12,145.87,146.18,144.82,145.74,24884478
2017-07-13,145.50,148.49,145.44,147.77,25199373
2017-07-14,147.97,149.33,147.33,149.04,20132061
2017-07-17,148.82,150.90,148.57,149.56,23793456


# You can define your own calendar using AbstractHolidayCalendar as shown below. USFederalHolidayCalendar is the only calendar available in pandas library and it serves as an example for those who want to write their own custom calendars. Here is the link for USFederalHolidayCalendar implementation https://github.com/pandas-dev/pandas/blob/master/pandas/tseries/holiday.py

# AbstractHolidayCalendar

In [18]:
from pandas.tseries.holiday import AbstractHolidayCalendar, nearest_workday, Holiday
class myCalendar(AbstractHolidayCalendar):
    rules = [
        Holiday('My Birth Day', month=4, day=15),#, observance=nearest_workday),# exclude this date
    ]
    
my_bday = CustomBusinessDay(calendar=myCalendar())
pd.date_range('4/1/2017','4/30/2017',freq=my_bday)

DatetimeIndex(['2017-04-03', '2017-04-04', '2017-04-05', '2017-04-06',
               '2017-04-07', '2017-04-10', '2017-04-11', '2017-04-12',
               '2017-04-13', '2017-04-14', '2017-04-17', '2017-04-18',
               '2017-04-19', '2017-04-20', '2017-04-21', '2017-04-24',
               '2017-04-25', '2017-04-26', '2017-04-27', '2017-04-28'],
              dtype='datetime64[ns]', freq='C')

In [21]:
class myCalendar(AbstractHolidayCalendar):
    rules = [
        Holiday('My Birth Day', month=4, day=15 ,observance=nearest_workday),# exclude this date
    ]
    
my_bday = CustomBusinessDay(calendar=myCalendar())

pd.date_range('4/1/2017','4/30/2017',freq=my_bday)
# if our holiday is at weekend day then nearest_workday will set nerest working day as holiday
# as here 15 is already saturday so it is also observed as holiday on 14

DatetimeIndex(['2017-04-03', '2017-04-04', '2017-04-05', '2017-04-06',
               '2017-04-07', '2017-04-10', '2017-04-11', '2017-04-12',
               '2017-04-13', '2017-04-17', '2017-04-18', '2017-04-19',
               '2017-04-20', '2017-04-21', '2017-04-24', '2017-04-25',
               '2017-04-26', '2017-04-27', '2017-04-28'],
              dtype='datetime64[ns]', freq='C')

# egypt weekend is friday and saturday

In [22]:
# so customize weekmask,, weekmask = 'Mon Tue Wed Thu Fri'

In [26]:
b = CustomBusinessDay(weekmask = 'Sun Mon Tue Wed Thu')
pd.date_range('7/1/2017','7/29/2017',freq=b) # as here 1,7,8..are fri or sat

DatetimeIndex(['2017-07-02', '2017-07-03', '2017-07-04', '2017-07-05',
               '2017-07-06', '2017-07-09', '2017-07-10', '2017-07-11',
               '2017-07-12', '2017-07-13', '2017-07-16', '2017-07-17',
               '2017-07-18', '2017-07-19', '2017-07-20', '2017-07-23',
               '2017-07-24', '2017-07-25', '2017-07-26', '2017-07-27'],
              dtype='datetime64[ns]', freq='C')

# imaginary world where only work on sundy monday

In [27]:
b = CustomBusinessDay(weekmask = 'Sun Mon')
pd.date_range('7/1/2017','7/29/2017',freq=b)

DatetimeIndex(['2017-07-02', '2017-07-03', '2017-07-09', '2017-07-10',
               '2017-07-16', '2017-07-17', '2017-07-23', '2017-07-24'],
              dtype='datetime64[ns]', freq='C')

In [28]:
# holiday in egypt are diff than us

In [29]:
b = CustomBusinessDay(weekmask = 'Sun Mon Tue Wed Thu',holidays = ['2017-07-04'])
pd.date_range('7/1/2017','7/29/2017',freq=b)

DatetimeIndex(['2017-07-02', '2017-07-03', '2017-07-05', '2017-07-06',
               '2017-07-09', '2017-07-10', '2017-07-11', '2017-07-12',
               '2017-07-13', '2017-07-16', '2017-07-17', '2017-07-18',
               '2017-07-19', '2017-07-20', '2017-07-23', '2017-07-24',
               '2017-07-25', '2017-07-26', '2017-07-27'],
              dtype='datetime64[ns]', freq='C')